### Optical Flow

In [ ]:
import cv2
import numpy as np

# Load the video
video_path = 'data/optical_flow/Soccer_Ball_Kick.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video was opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Read the first frame
ret, frame1 = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    exit()

# Convert the first frame to greyscale
prev_grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Create a mask image for drawing purposes
hsv_mask = np.zeros_like(frame1)
hsv_mask[..., 1] = 255


while True:
    # Read the next frame
    ret, frame2 = cap.read()
    if not ret:
        break

    # Convert the frame to greyscale
    grey = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Compute the dense optical flow using the Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_grey, grey, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute the magnitude and angle of the flow vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Set the hue according to the optical flow direction
    hsv_mask[..., 0] = angle * 180 / np.pi / 2

    # Set the value according to the optical flow magnitude (normalized)
    hsv_mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    # Convert HSV mask to BGR
    bgr_mask = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)

    # Overlay the original frame with the optical flow visualization
    output = cv2.addWeighted(frame2, 1, bgr_mask, 2, 0)

    # Display the result
    cv2.imshow('Optical Flow', cv2.resize(output, (1280, 720)))

    # Update the previous frame 
    prev_grey = grey

    # Exit if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()